<a href="https://colab.research.google.com/github/lana4zee/profit-and-sale-prediction/blob/main/Superstoreregression_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
!pip install pandas-profiling==2.7.1
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
from sklearn.tree import DecisionTreeRegressor
plt.style.use('fivethirtyeight')
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 252 kB 7.8 MB/s 
     |████████████████████████████████| 58 kB 7.3 MB/s 
     |████████████████████████████████| 679 kB 66.4 MB/s 
     |████████████████████████████████| 4.7 MB 47.0 MB/s 
     |████████████████████████████████| 296 kB 69.7 MB/s 
     |████████████████████████████████| 1.6 MB 62.2 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=20a52c27e738bda8889a87ae2c9f6e20f24154c04469ac38515bfa5a5396c56d
  Stored in directory: /root/.cache/pip/wheels/23/14/6e/4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
Successfully built htmlmin
  Attempting uninstall: pandas-profiling
    Found existing installation: pandas-profiling 1.4.1
    Uninstalling pandas-profiling-1.4.1:
      Successfully uninstalled pandas-profiling-1.4.1


# loading  dataset

In [ ]:
df=pd.read_csv('/content/SuperStoreOrders.csv')


In [ ]:
df.head()

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408,2,0.0,106.140,35.46,Medium,2011
1,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.036,9.72,Medium,2011
2,HU-2011-1220,1/1/2011,5/1/2011,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Office Supplies,Storage,"Tenex Box, Single Width",66,4,0.0,29.640,8.17,High,2011
3,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,...,Office Supplies,Paper,"Enermax Note Cards, Premium",45,3,0.5,-26.055,4.82,High,2011
4,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",114,5,0.1,37.770,4.70,Medium,2011


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        51290 non-null  object 
 1   order_date      51290 non-null  object 
 2   ship_date       51290 non-null  object 
 3   ship_mode       51290 non-null  object 
 4   customer_name   51290 non-null  object 
 5   segment         51290 non-null  object 
 6   state           51290 non-null  object 
 7   country         51290 non-null  object 
 8   market          51290 non-null  object 
 9   region          51290 non-null  object 
 10  product_id      51290 non-null  object 
 11  category        51290 non-null  object 
 12  sub_category    51290 non-null  object 
 13  product_name    51290 non-null  object 
 14  sales           51290 non-null  object 
 15  quantity        51290 non-null  int64  
 16  discount        51290 non-null  float64
 17  profit          51290 non-null 

In [ ]:
df.describe()

,quantity,discount,profit,shipping_cost,year
count,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000
mean,3.476545,0.142908,28.641740,26.375915,2012.777208
std,2.278766,0.212280,174.424113,57.296804,1.098931
min,1.000000,0.000000,-6599.978000,0.000000,2011.000000
25%,2.000000,0.000000,0.000000,2.610000,2012.000000
50%,3.000000,0.000000,9.240000,7.790000,2013.000000
75%,5.000000,0.200000,36.810000,24.450000,2014.000000
max,14.000000,0.850000,8399.976000,933.570000,2014.000000


In [ ]:
df.isnull().sum()

order_id          0
order_date        0
ship_date         0
ship_mode         0
customer_name     0
segment           0
state             0
country           0
market            0
region            0
product_id        0
category          0
sub_category      0
product_name      0
sales             0
quantity          0
discount          0
profit            0
shipping_cost     0
order_priority    0
year              0
dtype: int64

In [ ]:
df.sort_values(by='order_date', ascending=True)

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408,2,0.00,106.1400,35.46,Medium,2011
1,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120,3,0.10,36.0360,9.72,Medium,2011
2,HU-2011-1220,1/1/2011,5/1/2011,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Office Supplies,Storage,"Tenex Box, Single Width",66,4,0.00,29.6400,8.17,High,2011
3,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,...,Office Supplies,Paper,"Enermax Note Cards, Premium",45,3,0.50,-26.0550,4.82,High,2011
4,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",114,5,0.10,37.7700,4.70,Medium,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43964,IT-2014-5595800,9/9/2014,11/9/2014,Second Class,Jane Waco,Corporate,North Rhine-Westphalia,Germany,EU,Central,...,Office Supplies,Storage,"Rogers Box, Blue",22,1,0.10,-1.9470,3.05,Critical,2014
43965,US-2014-124779,9/9/2014,12/9/2014,First Class,Barry Französisch,Corporate,Texas,United States,US,Central,...,Office Supplies,Paper,Xerox 205,21,4,0.20,7.2576,3.04,High,2014
43966,US-2014-118794,9/9/2014,12/9/2014,First Class,Stefanie Holloman,Corporate,Santa Fe,Argentina,LATAM,South,...,Office Supplies,Labels,"Harbour Creations Color Coded Labels, 5000 Lab...",16,3,0.40,-1.6320,2.95,High,2014
43959,RS-2014-7450,9/9/2014,12/9/2014,Second Class,Karen Ferguson,Home Office,Bashkortostan,Russia,EMEA,EMEA,...,Office Supplies,Binders,"Ibico Binder, Durable",17,1,0.00,0.4800,4.47,Critical,2014


In [ ]:
# converting oder date and ship date into datetime
df['order_date'] = pd.to_datetime(df['order_date'])
df['ship_date'] = pd.to_datetime(df['ship_date'])
df.head()

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year,difference_order_entered_to_arrival
0,AG-2011-2040,2011-01-01,2011-06-01,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Storage,"Tenex Lockers, Blue",408,2,0.0,106.140,35.46,Medium,1970,151.0
1,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.036,9.72,Medium,1970,212.0
2,HU-2011-1220,2011-01-01,2011-05-01,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Storage,"Tenex Box, Single Width",66,4,0.0,29.640,8.17,High,1970,120.0
3,IT-2011-3647632,2011-01-01,2011-05-01,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,...,Paper,"Enermax Note Cards, Premium",45,3,0.5,-26.055,4.82,High,1970,120.0
4,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Furnishings,"Eldon Light Bulb, Duo Pack",114,5,0.1,37.770,4.70,Medium,1970,212.0


In [ ]:
df['year']=pd.DatetimeIndex(df['year']).year
df.head()

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year,difference_order_entered_to_arrival
0,AG-2011-2040,2011-01-01,2011-06-01,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Storage,"Tenex Lockers, Blue",408,2,0.0,106.140,35.46,Medium,1970,151.0
1,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.036,9.72,Medium,1970,212.0
2,HU-2011-1220,2011-01-01,2011-05-01,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Storage,"Tenex Box, Single Width",66,4,0.0,29.640,8.17,High,1970,120.0
3,IT-2011-3647632,2011-01-01,2011-05-01,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,...,Paper,"Enermax Note Cards, Premium",45,3,0.5,-26.055,4.82,High,1970,120.0
4,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Furnishings,"Eldon Light Bulb, Duo Pack",114,5,0.1,37.770,4.70,Medium,1970,212.0


In [ ]:
df['difference_order_entered_to_arrival']=(df['ship_date'] - df['order_date']) / np.timedelta64(1, 'D')
df.head()


,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year,difference_order_entered_to_arrival
0,AG-2011-2040,2011-01-01,2011-06-01,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Storage,"Tenex Lockers, Blue",408,2,0.0,106.140,35.46,Medium,1970,151.0
1,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.036,9.72,Medium,1970,212.0
2,HU-2011-1220,2011-01-01,2011-05-01,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Storage,"Tenex Box, Single Width",66,4,0.0,29.640,8.17,High,1970,120.0
3,IT-2011-3647632,2011-01-01,2011-05-01,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,...,Paper,"Enermax Note Cards, Premium",45,3,0.5,-26.055,4.82,High,1970,120.0
4,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Furnishings,"Eldon Light Bulb, Duo Pack",114,5,0.1,37.770,4.70,Medium,1970,212.0


In [ ]:
cleaned_df = df.drop(['product_name','country','customer_name','product_id','order_priority','sub_category','order_id','market','state','order_date','ship_date'], axis = 1)
  
print("segment  category: ",cleaned_df['segment'].unique())
print("region categories: ",cleaned_df['region'].unique())
print("Product: category",cleaned_df['category'].unique())
print("year:",cleaned_df['year'].unique())

print (cleaned_df)

segment  category:  ['Consumer' 'Home Office' 'Corporate']
region categories:  ['Africa' 'Oceania' 'EMEA' 'North' 'Canada' 'Southeast Asia' 'Central'
 'South' 'Caribbean' 'North Asia' 'East' 'West' 'Central Asia']
Product: category ['Office Supplies' 'Furniture' 'Technology']
year: [1970]
            ship_mode      segment   region         category sales  quantity  \
0      Standard Class     Consumer   Africa  Office Supplies   408         2   
1      Standard Class     Consumer  Oceania  Office Supplies   120         3   
2        Second Class     Consumer     EMEA  Office Supplies    66         4   
3        Second Class  Home Office    North  Office Supplies    45         3   
4      Standard Class     Consumer  Oceania        Furniture   114         5   
...               ...          ...      ...              ...   ...       ...   
51285  Standard Class    Corporate     West  Office Supplies    14         2   
51286  Standard Class     Consumer   Africa  Office Supplies     4    

In [ ]:
#dummy encoding categorical features
segment = pd.get_dummies(cleaned_df['segment'])
region = pd.get_dummies(cleaned_df['region'])
categories = pd.get_dummies(cleaned_df['category'])
years = pd.get_dummies(cleaned_df['year'])
ship_mode = pd.get_dummies(cleaned_df['ship_mode'])

#combining encoded features with orignal data
processed_results = pd.concat([cleaned_df, segment ,region,categories,years,ship_mode], axis=1, join='inner')
processed_results = processed_results.drop(['segment' ,'region','category','year','ship_mode'], axis = 1)
processed_results.head()

,sales,quantity,discount,profit,shipping_cost,difference_order_entered_to_arrival,Consumer,Corporate,Home Office,Africa,...,Southeast Asia,West,Furniture,Office Supplies,Technology,1970,First Class,Same Day,Second Class,Standard Class
0,408,2,0.0,106.140,35.46,151.0,1,0,0,1,...,0,0,0,1,0,1,0,0,0,1
1,120,3,0.1,36.036,9.72,212.0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1
2,66,4,0.0,29.640,8.17,120.0,1,0,0,0,...,0,0,0,1,0,1,0,0,1,0
3,45,3,0.5,-26.055,4.82,120.0,0,0,1,0,...,0,0,0,1,0,1,0,0,1,0
4,114,5,0.1,37.770,4.70,212.0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1


In [ ]:
X=processed_results.drop(['sales'],axis=1)
y=processed_results['sales']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)

# functions for model evaluation

In [ ]:
# model evaluation
def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    r2_score = metrics.r2_score(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))

    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('r2_score:', r2_score)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    r2_score = metrics.r2_score(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    return mae, mse, rmse,r2_score

# Extracting most important features from DecisionTreeRegressor

In [ ]:
# Initilizing Decision Tree Regression
dt_reg = DecisionTreeRegressor()

# Decision Tree Regression being fitted on the dataset
dt_reg.fit(X_train, y_train)

# Decision Tree - Test Predictions
test_pred_dt = dt_reg.predict(X_test)

# Decision Tree - Train Predictions
train_pred_dt = dt_reg.predict(X_train)


NameError: ignored

In [ ]:
l3=[]
l4=[]
for name, importance in zip(X.columns, dt_reg.feature_importances_):
    print(name, importance)
    l3.append(importance)
    l4.append(name)

In [ ]:
fig = px.bar(x=l4,y=l3, height=800)
fig.show()

# 1:training and evaluation using most important feature i.e Profit

In [ ]:
X_train1=X_train[['Profit']]
X_test1=X_test[['Profit']]

# Model 1:RandomForestRegressor

In [ ]:
rf_reg = RandomForestRegressor(random_state=1000)
rf_reg.fit(X_train1, y_train)
test_pred_rf = rf_reg.predict(X_test1)
train_pred_rf = rf_reg.predict(X_train1)


In [ ]:
print('========Random forest Model Results =========')
print('Test Dataset Score\n')

print_evaluate(y_test, test_pred_rf)

print('==================================\n')
print('Train Dataset Score\n')

print_evaluate(y_train, train_pred_rf)

# Comparison of actual vs predicted
comparison = pd.DataFrame({'True Values': y_test, 'Predicted Values': 
              test_pred_rf})
# Scatter Plot
plt.figure(figsize=(10,10))
sns.scatterplot(data=comparison, x="True Values", y="Predicted Values")
plt.show()

#actual vs predicted values comparison
plt.figure(figsize=(15,10))
plt.plot(
np.arange(len(y_test[0:200])) , y_test[0:200], "r-", label="Actual values" )
plt.plot( np.arange(len(test_pred_rf[0:200])) , test_pred_rf[0:200], "g-", label="Predicted values"
)
plt.scatter(
 np.arange(len(y_test[0:200])) , y_test[0:200], label="Actual values"
)
plt.scatter(
np.arange(len(test_pred_rf[0:200])) , test_pred_rf[0:200], label="Predicted values" )
plt.legend(['Actual', 'Predicted'])
plt.title('Actual vs Predicted for test data')

# Model 2:LinearRegression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train1,y_train)
test_pred_linear = lin_reg.predict(X_test1)
train_pred_linear = lin_reg.predict(X_train1)

In [ ]:
print('======== Linear regression Model Results =========')

print('Test Dataset Score\n')

print_evaluate(y_test, test_pred_linear)

print('==================================\n')
print('Train Dataset Score\n')

print_evaluate(y_train, train_pred_linear)

# Comparison of actual vs predicted
comparison = pd.DataFrame({'True Values': y_test, 'Predicted Values': 
              test_pred_linear})
# Scatter Plot
plt.figure(figsize=(10,10))
sns.scatterplot(data=comparison, x="True Values", y="Predicted Values")
plt.show()

#actual vs predicted values comparison
plt.figure(figsize=(15,10))
plt.plot(
np.arange(len(y_test[0:200])) , y_test[0:200], "r-", label="Actual values" )
plt.plot( np.arange(len(test_pred_linear[0:200])) , test_pred_linear[0:200], "g-", label="Predicted values"
)
plt.scatter(
 np.arange(len(y_test[0:200])) , y_test[0:200], label="Actual values"
)
plt.scatter(
np.arange(len(test_pred_linear[0:200])) , test_pred_linear[0:200], label="Predicted values" )
plt.legend(['Actual', 'Predicted'])
plt.title('Actual vs Predicted for test data')

# 2:training and evaluating using all the features

# Model 1:RandomForestRegressor

In [ ]:
rf_reg = RandomForestRegressor(random_state=1000)
rf_reg.fit(X_train, y_train)
test_pred_rf = rf_reg.predict(X_test)
train_pred_rf = rf_reg.predict(X_train)


In [ ]:
print('========Random forest Model Results =========')

print('Test Dataset Score\n')

print_evaluate(y_test, test_pred_rf)

print('==================================\n')
print('Train Dataset Score\n')

print_evaluate(y_train, train_pred_rf)

# Comparison of actual vs predicted
comparison = pd.DataFrame({'True Values': y_test, 'Predicted Values': 
              test_pred_rf})
# Scatter Plot
plt.figure(figsize=(10,10))
sns.scatterplot(data=comparison, x="True Values", y="Predicted Values")
plt.show()

#actual vs predicted values comparison
plt.figure(figsize=(15,10))
plt.plot(
np.arange(len(y_test[0:200])) , y_test[0:200], "r-", label="Actual values" )
plt.plot( np.arange(len(test_pred_rf[0:200])) , test_pred_rf[0:200], "g-", label="Predicted values"
)
plt.scatter(
 np.arange(len(y_test[0:200])) , y_test[0:200], label="Actual values"
)
plt.scatter(
np.arange(len(test_pred_rf[0:200])) , test_pred_rf[0:200], label="Predicted values" )
plt.legend(['Actual', 'Predicted'])
plt.title('Actual vs Predicted for test data')

# Model 2:LinearRegression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)
test_pred_linear = lin_reg.predict(X_test)
train_pred_linear = lin_reg.predict(X_train)

In [ ]:
print('======== Linear regression Model Results =========')

print('Test Dataset Score\n')

print_evaluate(y_test, test_pred_linear)

print('==================================\n')
print('Train Dataset Score\n')

print_evaluate(y_train, train_pred_linear)

# Comparison of actual vs predicted
comparison = pd.DataFrame({'True Values': y_test, 'Predicted Values': 
              test_pred_linear})
# Scatter Plot
plt.figure(figsize=(10,10))
sns.scatterplot(data=comparison, x="True Values", y="Predicted Values")
plt.show()

#actual vs predicted values comparison
plt.figure(figsize=(15,10))
plt.plot(
np.arange(len(y_test[0:200])) , y_test[0:200], "r-", label="Actual values" )
plt.plot( np.arange(len(test_pred_linear[0:200])) , test_pred_linear[0:200], "g-", label="Predicted values"
)
plt.scatter(
 np.arange(len(y_test[0:200])) , y_test[0:200], label="Actual values"
)
plt.scatter(
np.arange(len(test_pred_linear[0:200])) , test_pred_linear[0:200], label="Predicted values" )
plt.legend(['Actual', 'Predicted'])
plt.title('Actual vs Predicted for test data')

# Conclusion:
In this study, Superstore dataset was studied to predict the Sales based on certain features present in the dataset. In this study,we applied several experiments, one with taking profit as input to predict the sales and other by taking all features as input to predict the sales. RandomForestRegressor has performed better by taking all features as input for Superstore dataset with MAE of  96.217, and R2 score of 0.698 for the test data.